# Function Pointer

Sometimes, code must be flexible enough, to exchange different procedures or functions. Function pointer are a good way to handle such problems.

A function pointer is not the function/procedure itself, but the address in the memory of this function/procedure.
In order to access this address, the procedure is called without any brackets or arguments:

In [2]:
def EvaluateSum(SummandA, SummandB):
    return SummandA + SummandB

print('3+4=', EvaluateSum(3, 4))   
print('Address of this procedure in memory is: ', EvaluateSum)


3+4= 7
Address of this procedure in memory is:  <function EvaluateSum at 0x0000027AD5A9F4C0>


A simple example for using function pointer is shown below. A class CSignalFlowBlock is defined, which manages its own transfer function defined by a function pointer. Beside this, the signal flow block organises the transfer of the output data to the next processing step and finally it memorizes the last evaluation result.

In [10]:
class CSignalFlowBlock(object):
    
    def __init__(self, TransferFunctionPointer):
        self.__TransferFunctionPointer = TransferFunctionPointer
        self.__Output = None
        self.__LastOutput = None
        
    def RegisterOutput(self, NewOutput):
        self.__Output = NewOutput
        
    def ProcessNewData(self, NewData):
        self.__LastOutput = self.__TransferFunctionPointer(NewData)
        if self.__Output is not None:
            self.__Output.ProcessNewData(self.__LastOutput)
        return self.__LastOutput
    
    def GetLastOutput(self):
        return self.__LastOutput

In the following a set of simple transfer functions are defined:

In [11]:
import numpy as np

def Transferfunction1(x):
    return 3*x+1

def Transferfunction2(x):
    return x**2

def Transferfunction3(x):
    if np.sum(x) > 0:
        return 1.0
    else:
        return -1.0

A list of processing steps can be administrated easily by such a concept of signal flow blocks:

A list of signal flow blocks is established. All input data is given to the first signal flow block in the list. Each signal flow block organises the transfer function and the propagation of the data to the next signal flow block.

The final result can be extracted from the last signal flow block in the list.

In [12]:
ListOfSignalFlowBlocks = []
ListOfSignalFlowBlocks.append(CSignalFlowBlock(Transferfunction1))
ListOfSignalFlowBlocks.append(CSignalFlowBlock(Transferfunction2))
ListOfSignalFlowBlocks.append(CSignalFlowBlock(Transferfunction3))

for n in range(len(ListOfSignalFlowBlocks)-1):
    ListOfSignalFlowBlocks[n].RegisterOutput(ListOfSignalFlowBlocks[n+1])
    
ListOfSignalFlowBlocks[0].ProcessNewData(42)
print('the result of this chain of signal flow blocks is: ', ListOfSignalFlowBlocks[-1].GetLastOutput())

the result of this chain of signal flow blocks is:  1.0


## Exercise:
Implement a transfer function, which implements the neutral element (input equal to output).